In [6]:
import pynq
import numpy as np
import sys
import time


ol = pynq.Overlay('./design_1.bit')
mmio = ol.LEA_En_128_0.mmio

def ndarray_from_mmio(name, size, dtype):
    reginfo = ol.ip_dict['LEA_En_128_0']['registers'][name]
    addr_start = reginfo['address_offset'] // 4
    addr_end = addr_start + reginfo['size'] // 4
    mmio_array = mmio.array[addr_start:addr_end]
    mmio_array.dtype = np.uint32
    return mmio_array.reshape(size)


mmio_ciphertext = ndarray_from_mmio(
    'Memory_Cipher', size=4, dtype=np.uint32)
mmio_plaintext = ndarray_from_mmio('Memory_Plain', size=4, dtype=np.uint32)
mmio_key = ndarray_from_mmio('Memory_Key_K', size=4, dtype=np.uint32)


def LEA128Enc_fpga(ciphertext: np.ndarray, plaintext: np.ndarray, key: np.ndarray):
    mmio_plaintext[:] = plaintext
    mmio_key[:] = key
    mmio.write(0, 1)
    # 0x02 means DONE.
    while not (mmio.read(0) & 0x02):
        pass


# Encrypt a test vector using FPGA
# ISO/IEC 29192-2, p.49
key = np.array([0x3c2d1e0f, 0x78695a4b, 0xb4a59687, 0xf0e1d2c3])
plaintext = np.array([0x13121110, 0x17161514, 0x1b1a1918, 0x1f1e1d1c])
ciphertext = np.array([0x354ec89f, 0x18c6c628, 0xa7c73255, 0xfd8b6404])

LEA128Enc_fpga(mmio_ciphertext, plaintext, key)
ciphertext_fpga = mmio_ciphertext.copy()

# Check the result
for i in range(0, 4):
    if ciphertext[i] != ciphertext_fpga[i]:
        print("NG")
        for j in range(0, 4):
            print(str(j) + '  ' + format(ciphertext[j], '#08x') +
                  ' ' + format(ciphertext_fpga[j], '#08x'), end='')
            if j == i:
                print(' xxxxxxx', flush=True)
            else:
                print('', flush=True)
        sys.exit(1)

print("OK", flush=True)
np.set_printoptions(formatter={'int': '{:x}'.format})
print(ciphertext)

OK
[354ec89f 18c6c628 a7c73255 fd8b6404]


In [7]:
# Measure execution time.
NUM_OF_LOOPS = 1 << 17
i = 0
start_time = time.perf_counter_ns()
start_time_cpu = time.process_time_ns()
while i < NUM_OF_LOOPS:
    LEA128Enc_fpga(mmio_ciphertext, plaintext, key)
    i += 1
end_time_cpu = time.process_time_ns()
end_time = time.perf_counter_ns()
elapsed_time_ns = end_time - start_time
elapsed_time_cpu_ns = end_time_cpu - start_time_cpu
print('Num ' + str(NUM_OF_LOOPS))
print('Total ' + str(elapsed_time_ns) + ' ns')
print('CPU   ' + str(elapsed_time_cpu_ns) + ' ns')
print('Per 1 time ')
print('Total ' + '{} ns'.format((end_time -
      start_time) / NUM_OF_LOOPS))
print('CPU   ' + '{} ns'.format((end_time_cpu -
      start_time_cpu) / NUM_OF_LOOPS))

Num 131072
Total 8778107534 ns
CPU   8797672130 ns
Per 1 time 
Total 66971.64561462402 ns
CPU   67120.91163635254 ns
